In [1]:
import bcolz
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
inputs = bcolz.open('/data/poker/regress_on_bets_input.dat')[:]
outputs = bcolz.open('/data/poker/regress_on_bets_output.dat')[:]

cards_data = np.array(inputs[:,0].tolist())
bets_data = np.array(inputs[:,1].tolist()).reshape((inputs.shape[0], 3, 5,1))

In [7]:
from keras.layers import Input, Dense, Flatten, concatenate, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv2D, ZeroPadding2D
import keras.layers
from keras.optimizers import Nadam

cards_input = Input(shape=(4, 13, 4))
bets_input = Input(shape=(3,5,1))

cx = ZeroPadding2D(padding=(0, 2))(cards_input)
cx = Conv2D(30,(4,5), padding='same', activation='relu')(cx)
cx = Conv2D(20,(4,5), padding='valid', activation='relu')(cx)
cx = Flatten()(cx)

bx = Conv2D(30,(3,1), padding='valid', activation='relu')(bets_input)
bx = Conv2D(20,(1,1), padding='valid', activation='relu')(bx)
bx = Flatten()(bx)

tx = concatenate([cx, bx])
tx = Dense(100, activation='relu')(tx)
tx = Dropout(0.5)(tx)
tx = Dense(50, activation='relu')(tx)
tx = Dropout(0.5)(tx)
bet_output = Dense(1, name='bet_output')(tx)

model = Model([cards_input, bets_input], [bet_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.006)
model.compile(optimizer=opt,loss='mse')

In [15]:
model.fit([cards_data, bets_data], outputs, shuffle=True, validation_split=0.05, nb_epoch=5)

Train on 66821 samples, validate on 3517 samples
Epoch 1/5
  864/66821 [..............................] - ETA: 13s - loss: 1731.7827

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


66821/66821 [==============================] - 9s - loss: 1544.7134 - val_loss: 1636.9911
Epoch 2/5
66821/66821 [==============================] - 9s - loss: 1489.0639 - val_loss: 1444.8054
Epoch 3/5
66821/66821 [==============================] - 9s - loss: 1420.0014 - val_loss: 1425.8545
Epoch 4/5
66821/66821 [==============================] - 9s - loss: 1397.0225 - val_loss: 1392.9195
Epoch 5/5
66821/66821 [==============================] - 9s - loss: 1356.6478 - val_loss: 1458.9010


In [16]:
model.save_weights('/data/trained_models/poker/regres_on_bets_v1.h5')